In [1]:
import os
import shutil
import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image

subject ID: xxx

image number: xxx

gender: 0 - male 1 - famale

glasses: 0 - no 1 - yes

eye state: 0 - close 1 - open

reflections: 0 - none 1 - low 2 - high

lighting conditions/image quality: 0 - bad 1 - good

sensor type: 01 - RealSense SR300 640x480 02 - IDS Imaging, 1280x1024 03 - Aptina Imagin 752x480

In [59]:
Raw_DIR= r'C:\Users\Jayasuriya\Downloads\mrlEyes_2018_01'
for dirpath, dirname, filenames in os.walk(Raw_DIR):
    for i in tqdm([f for f in filenames if f.endswith('.png')]):
        if i.split('_')[4]=='0':
            shutil.copy(src=dirpath+'/'+i, dst=r'C:\Users\Jayasuriya\Downloads\mrlEyes_2018_01\prepared_data\closed eyes')
        
        elif i.split('_')[4]=='1':
            shutil.copy(src=dirpath+'/'+i, dst=r'C:\Users\Jayasuriya\Downloads\mrlEyes_2018_01\prepared_data\open eyes')

0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 10257/10257 [00:14<00:00, 715.69it/s]
0it [00:00, ?it/s]
  0%|          | 0/41946 [00:00<?, ?it/s]


SameFileError: 'C:\\Users\\Jayasuriya\\Downloads\\mrlEyes_2018_01\\prepared_data\\closed eyes/s0001_00001_0_0_0_0_0_01.png' and 'C:\\Users\\Jayasuriya\\Downloads\\mrlEyes_2018_01\\prepared_data\\closed eyes\\s0001_00001_0_0_0_0_0_01.png' are the same file

 MODEL TRAINING

In [2]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Data Augumentation
    


In [3]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [4]:
batchsize=8

In [5]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)

train_data= train_datagen.flow_from_directory(r'C:\Users\Jayasuriya\Downloads\mrlEyes_2018_01\prepared_data\train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training' )

validation_data= train_datagen.flow_from_directory(r'C:\Users\Jayasuriya\Downloads\mrlEyes_2018_01\prepared_data\train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='validation')

Found 63846 images belonging to 2 classes.
Found 15961 images belonging to 2 classes.


In [6]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'C:\Users\Jayasuriya\Downloads\mrlEyes_2018_01\prepared_data\test',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 5091 images belonging to 2 classes.


In [7]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 80, 80, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 39, 39, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 39, 39, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 39, 39, 32)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [9]:
model = Model(inputs = bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False
    

In [10]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau


In [11]:
checkpoint = ModelCheckpoint(os.path.join("models", "model.h5"),
                            monitor = 'val_loss', save_best_only = True, verbose = 3)

In [12]:
earlystop = EarlyStopping(monitor = 'val_loss', patience = 7, 
                          verbose= 3, restore_best_weights = True)

In [13]:
learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

In [14]:
callbacks = [checkpoint, earlystop, learning_rate]

In [15]:
model.compile(optimizer = 'Adam', 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

In [16]:
model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                    epochs=5)

C:\Users\Jayasuriya\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\keras\engine\training.py:1969: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
7980/7980 [==============================] - 327s 39ms/step - loss: 0.1915 - accuracy: 0.9262 - val_loss: 0.2772 - val_accuracy: 0.8939

Epoch 00001: val_loss improved from inf to 0.27723, saving model to models\model.h5
Epoch 2/5
7980/7980 [==============================] - 301s 38ms/step - loss: 0.1643 - accuracy: 0.9374 - val_loss: 0.2433 - val_accuracy: 0.8952

Epoch 00002: val_loss improved from 0.27723 to 0.24328, saving model to models\model.h5
Epoch 3/5
7980/7980 [==============================] - 302s 38ms/step - loss: 0.1571 - accuracy: 0.9398 - val_loss: 0.2582 - val_accuracy: 0.9086

Epoch 00003: val_loss did not improve from 0.24328
Epoch 4/5
7980/7980 [==============================] - 314s 39ms/step - loss: 0.1500 - accuracy: 0.9429 - val_loss: 0.2466 - val_accuracy: 0.9083

Epoch 00004: val_loss did not improve from 0.24328
Epoch 5/5
7980/7980 [==============================] - 319s 40ms/step - loss: 0.1448 - accuracy: 0.9443 - val_loss: 0.2495 - val_accuracy:

In [17]:
acc_tr, loss_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

C:\Users\Jayasuriya\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\keras\engine\training.py:2002: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


0.12333421409130096
0.9521974921226501


In [18]:
acc_vr, loss_vr = model.evaluate_generator(validation_data)
print(acc_vr)
print(loss_vr)

0.24709588289260864
0.9032642245292664


In [19]:
acc_test, loss_test = model.evaluate_generator(test_data)
print(acc_tr)
print(loss_tr)

0.12333421409130096
0.9521974921226501
